In [1]:
import pandas as pd
import numpy as np
import panel as pn
pn.extension('tabulator')

import hvplot.pandas

In [2]:
sample = pd.read_csv("C:\\Users\Lloyd\Documents\GitHub\DataAnalyticsFirst\GlobalWeatherRepository.csv")
sample[sample['country'] == 'Philippines']

,country,location_name,latitude,longitude,timezone,last_updated_epoch,last_updated,temperature_celsius,temperature_fahrenheit,condition_text,...,air_quality_PM2.5,air_quality_PM10,air_quality_us-epa-index,air_quality_gb-defra-index,sunrise,sunset,moonrise,moonset,moon_phase,moon_illumination
138,Philippines,Manila,14.6,120.98,Asia/Manila,1693302300,2023-08-29 17:45,27.0,80.6,Light rain,...,1.2,4.1,1,1,05:44 AM,06:10 PM,04:53 PM,03:28 AM,Waxing Gibbous,93
333,Philippines,Manila,14.6,120.98,Asia/Manila,1693364400,2023-08-30 11:00,31.0,87.8,Partly cloudy,...,2.2,5.9,1,1,05:44 AM,06:10 PM,05:46 PM,04:34 AM,Full Moon,98
528,Philippines,Manila,14.6,120.98,Asia/Manila,1693439100,2023-08-31 07:45,25.0,77.0,Light rain,...,3.0,6.8,1,1,05:44 AM,06:10 PM,05:46 PM,04:34 AM,Full Moon,98
723,Philippines,Manila,14.6,120.98,Asia/Manila,1693525500,2023-09-01 07:45,27.0,80.6,Heavy rain at times,...,2.8,6.3,1,1,05:44 AM,06:09 PM,06:32 PM,05:39 AM,Waning Gibbous,100
918,Philippines,Manila,14.6,120.98,Asia/Manila,1693611900,2023-09-02 07:45,27.0,80.6,Light rain,...,4.0,8.4,1,1,05:44 AM,06:08 PM,07:17 PM,06:40 AM,Waning Gibbous,99
1112,Philippines,Manila,14.6,120.98,Asia/Manila,1693698300,2023-09-03 07:45,27.0,80.6,Light rain,...,4.2,8.3,1,1,05:44 AM,06:08 PM,07:59 PM,07:39 AM,Waning Gibbous,94
1307,Philippines,Manila,14.6,120.98,Asia/Manila,1693783800,2023-09-04 07:30,25.0,77.0,Light rain,...,3.9,7.9,1,1,05:44 AM,06:07 PM,08:41 PM,08:36 AM,Waning Gibbous,88
1502,Philippines,Manila,14.6,120.98,Asia/Manila,1693870200,2023-09-05 07:30,28.0,82.4,Light rain,...,5.0,10.3,1,1,05:44 AM,06:06 PM,09:25 PM,09:33 AM,Waning Gibbous,79
1697,Philippines,Manila,14.6,120.98,Asia/Manila,1693955700,2023-09-06 07:15,28.0,82.4,Partly cloudy,...,10.2,23.9,1,1,05:44 AM,06:05 PM,10:10 PM,10:30 AM,Waning Gibbous,70
1892,Philippines,Manila,14.6,120.98,Asia/Manila,1694041200,2023-09-07 07:00,27.0,80.6,Partly cloudy,...,36.0,59.6,2,4,05:45 AM,06:05 PM,10:58 PM,11:28 AM,Last Quarter,60


In [3]:
isample = sample.interactive()

In [4]:
# Slider widget
slider = pn.widgets.IntSlider(name='Data slider', start=1, end=100, step=1, value=1)
slider

IntSlider(end=100, name='Data slider', start=1, value=1)

In [35]:
# Radio buttons for CO2 measures
radio_airquality = pn.widgets.RadioButtonGroup(
    name='Air Quality', 
    options=['air_quality_PM2.5', 'air_quality_PM10'],
    button_type='success'
)
countries = ['Philippines','United States of America','Canada','Germany','China','Japan','United Kingdom','Australia','India','Russia']

In [44]:
airquality_location_pipeline = (isample[(isample.moon_illumination <= slider) & (isample.country.isin(countries))]
    .groupby(['country','location_name','last_updated','air_quality_PM2.5'])['air_quality_PM10'].mean()
    .to_frame()
    .reset_index()
    .sort_values(by='country')  
    .reset_index(drop=True))

In [62]:
airquality_location_table = airquality_location_pipeline.pipe(pn.widgets.Tabulator, pagination='remote', page_size = 15, sizing_mode='stretch_width') 
airquality_location_table

In [43]:
airquality_location_scatterplot = airquality_location_pipeline.hvplot(x='country',y='air_quality_PM2.5',by='location_name',
                                                                    size=80,kind='scatter',
                                                                    alpha=0.2,
                                                                    legend=True,
                                                                    height=500,
                                                                    width=900,
                                                                    )

airquality_location_scatterplot

Traceback (most recent call last):
 File "C:\Users\Lloyd\anaconda3\Lib\site-packages\pyviz_comms\__init__.py", line 340, in _handle_msg
 self._on_msg(msg)
 File "C:\Users\Lloyd\anaconda3\Lib\site-packages\panel\viewable.py", line 471, in _on_msg
 doc.unhold()
 File "C:\Users\Lloyd\anaconda3\Lib\site-packages\bokeh\document\document.py", line 776, in unhold
 self.callbacks.unhold()
 File "C:\Users\Lloyd\anaconda3\Lib\site-packages\bokeh\document\callbacks.py", line 431, in unhold
 self.trigger_on_change(event)
 File "C:\Users\Lloyd\anaconda3\Lib\site-packages\bokeh\document\callbacks.py", line 408, in trigger_on_change
 invoke_with_curdoc(doc, event.callback_invoker)
 File "C:\Users\Lloyd\anaconda3\Lib\site-packages\bokeh\document\callbacks.py", line 443, in invoke_with_curdoc
 return f()
 ^^^
 File "C:\Users\Lloyd\anaconda3\Lib\site-packages\bokeh\util\callback_manager.py", line 185, in invoke
 callback(attr, old, new)
 File "C:\Users\Lloyd\anaconda3\Lib\site-packages\panel\reactive.py", line 469, in _comm_change
 state._handle_exception(e)
 File "C:\Users\Lloyd\anaconda3\Lib\site-packages\panel\io\state.py", line 436, in _handle_exception
 raise exception
 File "C:\Users\Lloyd\anaconda3\Lib\site-packages\panel\reactive.py", line 467, in _comm_change
 self._schedule_change(doc, comm)
 File "C:\Users\Lloyd\anaconda3\Lib\site-packages\panel\reactive.py", line 449, in _schedule_change
 self._change_event(doc)
 File "C:\Users\Lloyd\anaconda3\Lib\site-packages\panel\reactive.py", line 445, in _change_event
 self._process_events(events)
 File "C:\Users\Lloyd\anaconda3\Lib\site-packages\panel\reactive.py", line 384, in _process_events
 self.param.update(**self_events)
 File "C:\Users\Lloyd\anaconda3\Lib\site-packages\param\parameterized.py", line 1902, in update
 self_._batch_call_watchers()
 File "C:\Users\Lloyd\anaconda3\Lib\site-packages\param\parameterized.py", line 2063, in _batch_call_watchers
 self_._execute_watcher(watcher, events)
 File "C:\Users\Lloyd\anaconda3\Lib\site-packages\param\parameterized.py", line 2025, in _execute_watcher
 watcher.fn(*args, **kwargs)
 File "C:\Users\Lloyd\anaconda3\Lib\site-packages\panel\param.py", line 854, in _replace_pane
 new_object = self.eval(self.object)
 ^^^^^^^^^^^^^^^^^^^^^^
 File "C:\Users\Lloyd\anaconda3\Lib\site-packages\panel\param.py", line 813, in eval
 return eval_function(function)
 ^^^^^^^^^^^^^^^^^^^^^^^
 File "C:\Users\Lloyd\anaconda3\Lib\site-packages\panel\util\__init__.py", line 325, in eval_function
 return function(*args, **kwargs)
 ^^^^^^^^^^^^^^^^^^^^^^^^^
 File "C:\Users\Lloyd\anaconda3\Lib\site-packages\param\parameterized.py", line 407, in _depends
 return func(*args, **kw)
 ^^^^^^^^^^^^^^^^^
 File "C:\Users\Lloyd\anaconda3\Lib\site-packages\hvplot\interactive.py", line 369, in evaluate
 return evaluate_inner()
 ^^^^^^^^^^^^^^^^
 File "C:\Users\Lloyd\anaconda3\Lib\site-packages\hvplot\interactive.py", line 361, in evaluate_inner
 obj = self.eval()
 ^^^^^^^^^^^
 File "C:\Users\Lloyd\anaconda3\Lib\site-packages\hvplot\interactive.py", line 692, in eval
 obj = transform.apply(ds, keep_index=True, compute=False)
 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
 File "C:\Users\Lloyd\anaconda3\Lib\site-packages\holoviews\util\transform.py", line 770, in apply
 data = self._apply_fn(dataset, data, fn, fn_name, args,
 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
 File "C:\Users\Lloyd\anaconda3\Lib\site-packages\holoviews\util\transform.py", line 670, in _apply_fn
 raise e
 File "C:\Users\Lloyd\anaconda3\Lib\site-packages\holoviews\util\transform.py", line 664, in _apply_fn
 data = method(*args, **kwargs)
 ^^^^^^^^^^^^^^^^^^^^^^^
 File "C:\Users\Lloyd\anaconda3\Lib\site-packages\hvplot\plotting\core.py", line 92, in __call__
 return self._get_converter(x, y, kind, **kwds)(kind, x, y)
 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
 File "C:\Users\Lloyd\anaconda3\Lib\site-packages\hvplot\converter.py", line 1249, in __call__
 obj = method(x, y)
 ^^^^^^^^^^^^
 F

In [63]:
#Layout using Template
template = pn.template.FastListTemplate(
    title='Global Weather Report Update', 
    sidebar=[pn.pane.Markdown("# World Weather Repository ( Daily Updating )"), 
             pn.pane.Markdown("This dataset provides daily weather information for capital cities around the world. Unlike forecast data, this dataset offers a comprehensive set of features that reflect the current weather conditions around the world."), 
             pn.pane.PNG('https://storage.googleapis.com/kaggle-datasets-images/3678699/6383099/61a1c18c08506e236bf0cd0394c09110/dataset-cover.jpg?t=2023-08-29-10-36-14', sizing_mode='scale_both'),
             pn.pane.Markdown("## Settings")],
    main=[pn.Row(pn.Column(airquality_location_scatterplot.panel(width=600), margin=(0,25)),
                pn.Row(pn.Column(airquality_location_table.panel(width=600))))],
    accent_base_color="#88d8b0",
    header_background="#444444",
)
template.show()
template.servable();

Launching server at http://localhost:59566
